In [1]:
import sys
sys.path.append("D:\\WETrak")

import os
import os.path as path
import numpy as np

from scipy import interpolate
from scipy.signal import savgol_filter
from scipy.signal import iirfilter,lfilter

os.environ['KMP_DUPLICATE_LIB_OK']='True'

## 1. Helper Function for GT

### 1.1 Load and Write of GT

In [2]:
def parse_leap_all_finger(leap_data,finger_type_list):
    # (one file)
    pitch_yaw_values = []
    for i,per_leap_data in enumerate(leap_data):
        pitch_yaw_values_by_finger = []
        if "bone_angles" not in per_leap_data["left_hand"]:
            continue
            
        for finger_keys in finger_type_list:
            finger = per_leap_data["left_hand"]["bone_angles"][finger_keys]
            for bone_keys in ["Distal", "Intermediate", "Proximal"]:
                pitch_yaw_values_by_finger.append(finger[bone_keys]["Pitch"])
            pitch_yaw_values_by_finger.append(finger["Proximal"]["Yaw"])
    
        pitch_yaw_values.append(pitch_yaw_values_by_finger)
    
    pitch_yaw_values = np.array(pitch_yaw_values)
    new_pitch_yaw_values = pitch_yaw_values.reshape((pitch_yaw_values.shape[0],len(finger_type_list),4))
    
    return new_pitch_yaw_values

In [3]:
def compose_leap_all_finger(leap_data,finger_type_list):
    result = []
    for i in range(leap_data.shape[0]):
        info = {}
        info['left_hand'] = {}
        info['left_hand']['bone_angles'] = {}
        for j,finger_type in enumerate(finger_type_list):
            info['left_hand']['bone_angles'][finger_type] = {}
            for k,bone_keys in enumerate(["Distal","Intermediate","Proximal"]):
                info['left_hand']['bone_angles'][finger_type][bone_keys] = {}
                info['left_hand']['bone_angles'][finger_type][bone_keys]['Pitch'] = leap_data[i,j,k]
                if bone_keys == "Proximal":
                    info['left_hand']['bone_angles'][finger_type][bone_keys]['Yaw'] = leap_data[i,j,k+1]
                    
        result.append(info)
    
    return result

### 1.2 Transformation and Remove Outliers of GT

In [4]:
def constraints_angle_by_finger(data,finger_type):
    if finger_type == "Thumb":
        start = [-35,-20,-40,-20]
        end = [60,60,45,50]
        
    elif finger_type == "Index":
        start = [-5,-3,-40,-30]
        end = [90,110,90,30]
        
    elif finger_type == "Middle":
        start = [-5,-3,-40,-22.5]
        end = [90,110,90,22.5]
        
    elif finger_type == "Ring":
        start = [-5,-3,-40,-22.5]
        end = [90,120,90,22.5]
    
    elif finger_type == "Pinky":
        start = [-5,-3,-40,-25]
        end = [90,135,90,25]
        
    delta = -np.array(start)    
    data += delta
    removed_idx = []
    
    for i in [0,1,2,3]:
        lower_bound = start[i] + delta[i]
        upper_bound = end[i] + delta[i]
                   
        removed_idx += ((np.where((data[:,i] < lower_bound) | (data[:,i] > upper_bound))[0]).tolist())
        
    return data,removed_idx

### 1.3 Interpolate of GT

In [5]:
def interpolate_leap_all_finger(leap_data,leap_length,emg_length):
    x = np.linspace(0,leap_length-1,leap_length)
    x_new = np.linspace(0,leap_length-1,emg_length)
    kind = 'linear'
    
    new_data = np.zeros((emg_length,leap_data.shape[1],leap_data.shape[2]))
    for dim in range(leap_data.shape[2]):
        # Loop over each sample in this dimension
        for sample in range(leap_data.shape[1]):
            # Create interpolation function for this column/sample
            f = interpolate.interp1d(x, leap_data[:,sample,dim], kind)
            # Interpolate new values for this column/sample
            new_data[:, sample, dim] = f(x_new)
    
    return new_data

### 1.4 Smooth GT

In [6]:
def default_min_max(finger_type):
    if finger_type == "Thumb":
        start = [-35,-20,-40,-20]
        end = [60,60,45,50]
        
    elif finger_type == "Index":
        start = [-5,-3,-40,-30]
        end = [90,110,90,30]
        
    elif finger_type == "Middle":
        start = [-5,-3,-40,-22.5]
        end = [90,110,90,22.5]
        
    elif finger_type == "Ring":
        start = [-5,-3,-40,-22.5]
        end = [90,120,90,22.5]
    
    elif finger_type == "Pinky":
        start = [-5,-3,-40,-25]
        end = [90,135,90,25]
        
    delta = -np.array(start)    
    min_vals = []
    max_vals = []
    for i in [0,1,2,3]:
        lower_bound = start[i] + delta[i]
        upper_bound = end[i] + delta[i]
        
        min_vals.append(lower_bound)
        max_vals.append(upper_bound)
    
    return min_vals,max_vals

In [7]:
def smooth_gt(leap_data_list,finger_type_list):
    min_vals_list = []
    max_vals_list = []
    for finger_type in finger_type_list:
        min_vals,max_vals = default_min_max(finger_type)
        min_vals_list += min_vals
        max_vals_list += max_vals
    max_vals_list = [x - 0.1 for x in max_vals_list] #为了让smooth后的值包括在default范围内
    
    new_leap_data_list = []
    for idx,leap_data in enumerate(leap_data_list):    
        smoothAngles = np.zeros_like(leap_data)
        # window_length即窗口长度
        # 取值为奇数且不能超过len(x)。
        # 它越大，则平滑效果越明显；越小，则更贴近原始曲线。
        # polyorder为多项式拟合的阶数。
        # 它越小，则平滑效果越明显；越大，则更贴近原始曲线。
        windowsize = 199
        poly_deg = 6

        for i in range(leap_data.shape[1]):
            smoothAngles[:,i] = savgol_filter(leap_data[:,i], windowsize, poly_deg)
        
        smoothAngles = np.clip(smoothAngles, min_vals_list, max_vals_list)    
        new_leap_data_list.append(smoothAngles)
        
    return new_leap_data_list

### 1.5 Normalization of GT

In [8]:
def gt_normalize(leap_data_list,finger_type_list):
    min_vals_list = []
    max_vals_list = []
    new_leap_data_list = []
    
    for leap_data in leap_data_list:
        min_vals = np.min(leap_data, axis=0) #每一段2s数据的最小值
        max_vals = np.max(leap_data, axis=0) #每一段2s数据的最大值
        # 归一化后的leap data
        new_leap_data = (leap_data - min_vals) / (max_vals - min_vals)
        
        min_vals_list.append(min_vals) #所有条数据的最小值
        max_vals_list.append(max_vals) #所有条数据的最大值
        new_leap_data_list.append(new_leap_data)
    
    return np.array(min_vals_list),np.array(max_vals_list),np.array(new_leap_data_list)

## 2. Helper function for EMG

### 2.1 Filter of EMG

In [9]:
def Implement_Notch_Filter(time, band, freq, order, filter_type, data):
    
    fs   = 1/time
    nyq  = fs/2.0
    low  = freq - band/2.0
    high = freq + band/2.0
    low  = low/nyq
    high = high/nyq
    b, a = iirfilter(order, [low, high],  btype='bandstop',
                     analog=False, ftype=filter_type)
    filtered_data = lfilter(b, a, data)
    
    return filtered_data

In [10]:
def emg_z_norm(emg_data_list):
    norm_emg_data_list = []
    for emg_data_raw in emg_data_list:
        norm_emg_data = []

        for i in range(emg_data_raw.shape[1]):
            emg_data = emg_data_raw[:,i]
            # normalize z-score 
            mu = np.mean(emg_data,axis=0)
            sigma = np.std(emg_data,axis=0)
            emg_data = (emg_data - mu)/sigma

            norm_emg_data.append(emg_data)
        
        norm_emg_data_list.append(np.array(norm_emg_data).T)
    
    return norm_emg_data_list

## 3. Helper Function for Both GT and EMG

### 3.1 Split Train and Test Datasets

In [11]:
def make_dir(d):
    if not os.path.exists(d):
        os.makedirs(d)
    return d

In [12]:
def split_data(emg_data,leap_data,split_length,train_ratio,val_ratio,test_ratio):
    assert len(emg_data) == len(leap_data), "The lengths of emg_data and leap_data are same."
    
    split_num = int(len(emg_data)/split_length)
    
    split_emg_data = np.split(emg_data[:split_num*split_length,:], [i for i in range(split_length,split_num*split_length,split_length)]) # list
    split_leap_data = np.split(leap_data[:split_num*split_length,:], [i for i in range(split_length,split_num*split_length,split_length)]) 
    
    total_samples = len(split_emg_data)
    train_idx = int(total_samples * train_ratio)
    val_idx = int(total_samples * (train_ratio + val_ratio))
    
    emg_train = split_emg_data[:train_idx]
    leap_train = split_leap_data[:train_idx]

    emg_val = split_emg_data[train_idx:val_idx]
    leap_val = split_leap_data[train_idx:val_idx]

    emg_test = split_emg_data[val_idx:]
    leap_test = split_leap_data[val_idx:]
    
    if val_ratio == 0:
        return emg_train,leap_train,emg_test,leap_test
    else:
        return emg_train,leap_train,emg_val,leap_val,emg_test,leap_test